# 实现Transformer 的学习、推测及判定依据的可视化

将从 IMDb 的 DataLoader 中读取数据，然后通过实现的 Transformer 模型进行学习，对电影的评论文章（英文）中的评价内容是属于正面评价还是负面评价进行判定处理。此外，本节还将在判定时是根据哪个单词进行判断的依据，Self-Attention 实现可视化处理。

#  学习目标

1.	掌握编程实现 Transformer 的学习的方法
2.	掌握对 Transformer 进行判定时的 Attention 进行可视化处理的方法。


# 事前准备

·使用文件夹“utils”中的函数或类

In [1]:
# 导入软件包
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

In [2]:
# 设定随机数的种子，
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

#准备DataLoader 和Transformer 模型

In [3]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT
            
# 载入数据
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(max_length=256, batch_size=64)

# 集中保存到字典对象中
dataloaders_dict = {"train": train_dl, "val": val_dl}

#创建网络模型

In [4]:
from utils.transformer import TransformerClassification

# 构建模型
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# 定义网络的初始化操作
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
       #Liner层的初始化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# 设置为训练模式
net.train()

# 执行TransformerBlock模块的初始化操作
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('网络设置完毕')

网络设置完毕


# 损失函数与最优化算法

In [5]:
# 设置损失函数
criterion = nn.CrossEntropyLoss()
# 先计算nn.LogSoftmax，再计算nn.NLLLoss(negative log likelihood loss)

# 设置最优化算法
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

#训练与验证函数的编写和执行

In [6]:
# 创建用于训练模型的函数
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 确认是否能够使用GPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用的设备：", device)
    print('-------start-------')
    # 将网络载入GPU中
    net.to(device)

    # 如果网络结构比较固定，则开启硬件加速
    torch.backends.cudnn.benchmark = True

    # epoch循环
    for epoch in range(num_epochs):
        # 以epoch为单位进行训练和验证的循环
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 将模型设为训练模式
            else:
                net.eval()   # 将模型设为训练模式

            epoch_loss = 0.0  # epoch的损失
            epoch_corrects = 0  # epoch的准确率

            # 从数据加载器中读取小批次数据的循环
            for batch in (dataloaders_dict[phase]):
                # batch是Text和Lable的字典对象

                # 如果GPU可以使用，则将数据输送到GPU中
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # 标签

                # 初始化optimizer
                optimizer.zero_grad()

                # 初始化optimizer
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 在单词ID中'<pad>': 1
                    input_mask = (inputs != input_pad)

                    # 输入Transformer中
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 计算损失值

                    _, preds = torch.max(outputs, 1)   # 对标签进行预测

                    # 训练时进行反向传播
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 计算结果
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 更新正确答案的合计数量
                    epoch_corrects += torch.sum(preds == labels.data)

            # 每轮epoch的loss和准确率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss, epoch_acc))

    return net

In [7]:
# 执行学习和验证处理所需时间约为15分钟
num_epochs = 10
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用的设备： cuda:0
-------start-------
Epoch 1/10 | train |  Loss: 0.5938 Acc: 0.6679
Epoch 1/10 |  val  |  Loss: 0.4385 Acc: 0.8086
Epoch 2/10 | train |  Loss: 0.4392 Acc: 0.8012
Epoch 2/10 |  val  |  Loss: 0.4051 Acc: 0.8190
Epoch 3/10 | train |  Loss: 0.4035 Acc: 0.8186
Epoch 3/10 |  val  |  Loss: 0.3844 Acc: 0.8274
Epoch 4/10 | train |  Loss: 0.3846 Acc: 0.8317
Epoch 4/10 |  val  |  Loss: 0.3880 Acc: 0.8292
Epoch 5/10 | train |  Loss: 0.3731 Acc: 0.8372
Epoch 5/10 |  val  |  Loss: 0.3584 Acc: 0.8422
Epoch 6/10 | train |  Loss: 0.3633 Acc: 0.8388
Epoch 6/10 |  val  |  Loss: 0.3565 Acc: 0.8446
Epoch 7/10 | train |  Loss: 0.3572 Acc: 0.8427
Epoch 7/10 |  val  |  Loss: 0.3495 Acc: 0.8498
Epoch 8/10 | train |  Loss: 0.3451 Acc: 0.8508
Epoch 8/10 |  val  |  Loss: 0.3511 Acc: 0.8464
Epoch 9/10 | train |  Loss: 0.3366 Acc: 0.8553
Epoch 9/10 |  val  |  Loss: 0.3478 Acc: 0.8484
Epoch 10/10 | train |  Loss: 0.3306 Acc: 0.8587
Epoch 10/10 |  val  |  Loss: 0.3332 Acc: 0.8560


# 求测试数据的正确率

In [8]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()  # 将模型转换为验证模式
net_trained.to(device)

epoch_corrects = 0  # epoch的正确答案数

for batch in (test_dl):  # test数据的DataLoader
    
    # batch是Text和Label的字典对象
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # 标签

    # 正向传播计算
    with torch.set_grad_enabled(False):

        # mask作成
        input_pad = 1  # 在单词ID中'<pad>': 1
        input_mask = (inputs != input_pad)

        # 输入Transformer中
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # 对标签进行预测

        # 结果的计算
        # 更新正确答案的合计数量
        epoch_corrects += torch.sum(preds == labels.data)

# 准确率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('总数{}个的测试数据的准确率：{:.4f}'.format(len(test_dl.dataset),epoch_acc))

总数25000个的测试数据的准确率：0.8498


# 通过Attention 的可视化寻找判断依据



In [9]:
# 编写生成HTML的函数
def highlight(word, attn):
    "如果Attention的值较大，则函数将输出较深的红色作为文字背景的HTML"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "生成HTML数据"

    # 取出index的结果
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # 标签
    pred = preds[index]  # 预测

    # index的Attention的提取和归一化
    attens1 = normlized_weights_1[index, 0, :]  # 第0个<cls>的Attention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 第0个<cls>的Attention
    attens2 /= attens2.max()

    # 将标签和预测结果替换成文字信息
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 生成用于显示的HTML
    html = '正确答案 ：{}<br>推论标签 ：{}<br><br>'.format(label_str, pred_str)

    # 第一段Attention
    html += '[对TransformerBlock的第一段Attention进行可视化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 第二段Attention
    html += '[对TransformerBlock的第二段Attention进行可视化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html

In [10]:
from IPython.display import HTML

# 使用Transformer进行处理

# 准备好小批次
batch = next(iter(test_dl))

# 如果GPU可以使用，则将数据输送到GPU中
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # 标签

# 创建mask
input_pad = 1  # 在单词ID中'<pad>': 1
input_mask = (inputs != input_pad)

# 输入数据到Transformer
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # 对标签进行预测


index = 3  # 指定需要输出的数据
html_output = mk_html(index, batch, preds, normlized_weights_1, 
                      normlized_weights_2, TEXT)  # 生成HTML
HTML(html_output)  # 使用HTML格式输出

In [11]:
index = 61  # 需要输出的数据
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # 生成HTML
HTML(html_output)  # 用HTML格式输出